In [1]:
!git clone https://github.com/ggerganov/llama.cpp


Cloning into 'llama.cpp'...
remote: Enumerating objects: 18227, done.
remote: Counting objects: 100% (18227/18227), done.
remote: Compressing objects: 100% (5404/5404), done.
remote: Total 18227 (delta 12740), reused 18114 (delta 12684), pack-reused 0
Receiving objects: 100% (18227/18227), 20.83 MiB | 911.00 KiB/s, done.
Resolving deltas: 100% (12740/12740), done.


In [22]:

import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [12]:

train_data=pd.read_csv("/mnt/scratch/ponrames/WIDS/data/training.csv")
train_data

,patient_id,patient_race,payer_type,patient_state,patient_zip3,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,...,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D
0,475714,NaN,MEDICAID,CA,924,84,F,NaN,C50919,Malignant neoplasm of unsp site of unspecified...,...,12.871429,22.542857,10.100000,27.814286,11.200000,3.500000,52.237210,8.650555,18.606528,1
1,349367,White,COMMERCIAL,CA,928,62,F,28.49,C50411,Malig neoplm of upper-outer quadrant of right ...,...,8.957576,10.109091,8.057576,30.606061,7.018182,4.103030,42.301121,8.487175,20.113179,1
2,138632,White,COMMERCIAL,TX,760,43,F,38.09,C50112,Malignant neoplasm of central portion of left ...,...,11.253333,9.663333,3.356667,31.394915,15.066667,7.446667,40.108207,7.642753,14.839351,1
3,617843,White,COMMERCIAL,CA,926,45,F,NaN,C50212,Malig neoplasm of upper-inner quadrant of left...,...,8.845238,8.688095,5.280952,27.561905,4.404762,4.809524,42.070075,7.229393,15.894123,0
4,817482,NaN,COMMERCIAL,ID,836,55,F,NaN,1749,"Malignant neoplasm of breast (female), unspeci...",...,15.276000,11.224000,1.946000,26.170213,12.088000,13.106000,41.356058,4.110749,11.722197,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12901,674178,White,NaN,OH,436,50,F,32.11,C50411,Malig neoplm of upper-outer quadrant of right ...,...,17.400000,23.600000,0.864706,19.841176,6.300000,6.247059,38.753055,8.068682,21.140731,1
12902,452909,NaN,COMMERCIAL,CA,945,50,F,NaN,C50912,Malignant neoplasm of unspecified site of left...,...,11.243210,7.837037,5.411250,34.700000,3.845679,5.671605,36.469947,6.265266,10.728732,1
12903,357486,NaN,COMMERCIAL,CA,926,61,F,29.24,C50912,Malignant neoplasm of unspecified site of left...,...,8.845238,8.688095,5.280952,27.561905,4.404762,4.809524,42.070075,7.229393,15.894123,1
12904,935417,NaN,NaN,NY,112,37,F,31.00,1749,"Malignant neoplasm of breast (female), unspeci...",...,10.194737,18.642105,14.173684,42.502632,6.392105,1.755263,37.722740,7.879795,27.496367,0


In [11]:
test_data=pd.read_csv('/mnt/scratch/ponrames/WIDS/data/test.csv')
test_data

,patient_id,patient_race,payer_type,patient_state,patient_zip3,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,...,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02
0,573710,White,MEDICAID,IN,467,54,F,NaN,C50412,Malig neoplasm of upper-outer quadrant of left...,...,3.564516,13.996774,7.985484,0.969355,24.955357,10.838710,8.080645,38.724876,7.947165,11.157161
1,593679,NaN,COMMERCIAL,FL,337,52,F,NaN,C50912,Malignant neoplasm of unspecified site of left...,...,10.261538,16.020513,13.602564,2.836842,23.952632,10.579487,9.302564,36.918257,7.838973,13.599985
2,184532,Hispanic,MEDICAID,CA,917,61,F,NaN,C50911,Malignant neoplasm of unsp site of right femal...,...,47.726087,9.895652,10.515217,12.745652,32.530435,7.263043,3.810870,47.310325,9.595719,20.084231
3,447383,Hispanic,MEDICARE ADVANTAGE,CA,917,64,F,NaN,C50912,Malignant neoplasm of unspecified site of left...,...,47.726087,9.895652,10.515217,12.745652,32.530435,7.263043,3.810870,47.310325,9.595719,20.084231
4,687972,Black,NaN,CA,900,40,F,23.00,C50412,Malig neoplasm of upper-outer quadrant of left...,...,45.526154,11.901538,20.760000,14.737500,30.709375,10.341538,3.030769,41.186992,11.166898,21.644261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5787,977076,White,NaN,KY,404,63,F,29.60,C50411,Malig neoplm of upper-outer quadrant of right ...,...,1.243478,20.404348,20.813636,0.350000,30.152174,6.473913,5.908696,39.947326,7.622672,9.154618
5788,922960,White,NaN,IA,507,69,F,NaN,C50912,Malignant neoplasm of unspecified site of left...,...,6.175000,16.675000,15.900000,2.800000,16.800000,5.475000,6.875000,35.825340,7.610534,9.712786
5789,759690,NaN,MEDICARE ADVANTAGE,WA,980,84,F,28.28,C50411,Malig neoplm of upper-outer quadrant of right ...,...,9.511321,9.752830,6.432075,5.094340,31.275472,5.309434,5.807547,36.618644,4.939852,23.393650
5790,911717,NaN,COMMERCIAL,OK,740,58,F,NaN,1749,"Malignant neoplasm of breast (female), unspeci...",...,4.374627,15.544776,16.603030,0.513636,25.877273,14.926866,7.600000,39.832235,8.030925,9.769358


In [23]:
start=0
end=15

#print(axes)
na_df=train_data.isna()
lst=[]
for i in range(6):
    lst.append(na_df.iloc[:,start:end])
    start=end
    end+=15

    if(i==5):
        end=len(na_df)
    
print(len(lst)) 

6


In [ ]:
def plot():
    fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(24, 96))
    axes=axes.flatten().tolist()
    for i in range(len(axes)):
        ax = axes[i]
        sns.heatmap(lst[i], annot=True,cmap='viridis', ax=ax)
        #print(na_df.iloc[:,start:end])
        ax.set_title(f'Subplot {i+1}')
    plt.tight_layout()

    plt.show()

In [36]:
len(train_data)

12906

In [30]:
cols=list(train_data.columns)
cols

['patient_id',
 'patient_race',
 'payer_type',
 'patient_state',
 'patient_zip3',
 'patient_age',
 'patient_gender',
 'bmi',
 'breast_cancer_diagnosis_code',
 'breast_cancer_diagnosis_desc',
 'metastatic_cancer_diagnosis_code',
 'metastatic_first_novel_treatment',
 'metastatic_first_novel_treatment_type',
 'Region',
 'Division',
 'population',
 'density',
 'age_median',
 'age_under_10',
 'age_10_to_19',
 'age_20s',
 'age_30s',
 'age_40s',
 'age_50s',
 'age_60s',
 'age_70s',
 'age_over_80',
 'male',
 'female',
 'married',
 'divorced',
 'never_married',
 'widowed',
 'family_size',
 'family_dual_income',
 'income_household_median',
 'income_household_under_5',
 'income_household_5_to_10',
 'income_household_10_to_15',
 'income_household_15_to_20',
 'income_household_20_to_25',
 'income_household_25_to_35',
 'income_household_35_to_50',
 'income_household_50_to_75',
 'income_household_75_to_100',
 'income_household_100_to_150',
 'income_household_150_over',
 'income_household_six_figure',


In [45]:
for col in cols:
    print(f"{col} , Unique {len(train_data[col].unique())} , Nan {(train_data[col].isna().sum()*100)/12906:,.2f}% \n")

patient_id , Unique 12906 , Nan 0.00% 

patient_race , Unique 6 , Nan 49.47% 

payer_type , Unique 4 , Nan 13.97% 

patient_state , Unique 51 , Nan 0.40% 

patient_zip3 , Unique 739 , Nan 0.00% 

patient_age , Unique 67 , Nan 0.00% 

patient_gender , Unique 1 , Nan 0.00% 

bmi , Unique 1343 , Nan 69.46% 

breast_cancer_diagnosis_code , Unique 50 , Nan 0.00% 

breast_cancer_diagnosis_desc , Unique 50 , Nan 0.00% 

metastatic_cancer_diagnosis_code , Unique 43 , Nan 0.00% 

metastatic_first_novel_treatment , Unique 3 , Nan 99.81% 

metastatic_first_novel_treatment_type , Unique 2 , Nan 99.81% 

Region , Unique 5 , Nan 0.40% 

Division , Unique 10 , Nan 0.40% 

population , Unique 739 , Nan 0.01% 

density , Unique 739 , Nan 0.01% 

age_median , Unique 729 , Nan 0.01% 

age_under_10 , Unique 719 , Nan 0.01% 

age_10_to_19 , Unique 726 , Nan 0.01% 

age_20s , Unique 725 , Nan 0.01% 

age_30s , Unique 726 , Nan 0.01% 

age_40s , Unique 716 , Nan 0.01% 

age_50s , Unique 721 , Nan 0.01% 

age

In [47]:
demograpic_cols=['patient_race']
age_cols=['age_median','age_under_10',"age_under_10","age_10_to_19","age_20s","age_30s","age_40s","age_50s" ,"age_60s","age_70s" ]
race=["patient_race","race_white","race_black","race_asian","race_native","race_pacific","race_multiple","hispanic"]
education=["education_less_highschool","education_highschool","education_some_college","education_bachelors","education_graduate","education_college_or_above","education_stem_degree"]

work=["labor_force_participation","unemployment_rate","self_employed","farmer"]

,race_white,race_black,race_asian,race_native,race_pacific,race_multiple,hispanic
race_white,1.000000,-0.633767,-0.589537,-0.070888,-0.164892,-0.468582,-0.520393
race_black,-0.633767,1.000000,-0.049769,-0.130712,-0.084646,-0.125012,-0.094144
race_asian,-0.589537,-0.049769,1.000000,-0.053142,0.270820,0.362008,0.337937
race_native,-0.070888,-0.130712,-0.053142,1.000000,0.024260,0.129120,0.068185
race_pacific,-0.164892,-0.084646,0.270820,0.024260,1.000000,0.297562,0.077708
race_multiple,-0.468582,-0.125012,0.362008,0.129120,0.297562,1.000000,0.867806
hispanic,-0.520393,-0.094144,0.337937,0.068185,0.077708,0.867806,1.000000
